Install Necessary Packages
Install Packages in Jupyter Notebook **bold text**
Open your Jupyter Notebook and run the following commands to install the required packages:

In [ ]:
!pip install llama-index-llms-ollama
!pip install llama-index-embeddings-huggingface
!pip install llama-index
!pip install langchain
!pip install transformers
!pip install huggingface_hub
!pip install --upgrade llama-index
!pip install --upgrade langchain
!pip install --upgrade transformers
!pip install --upgrade huggingface_hub
!pip install --upgrade llama-index-llms-ollama llama-index-embeddings-huggingface

**Define Settings and Load Documents**
Define Settings

In [ ]:
import json
import logging
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, Settings
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.llms.ollama import Ollama
from llama_index.readers.file import PDFReader

**Load Documents**

In [ ]:
# Read input JSON file to get the PDF path
with open('input.json', 'r') as file:
    input_data = json.load(file)
    pdf_directory = input_data['pdf_directory']

# PDF Reader with SimpleDirectoryReader
parser = PDFReader()
file_extractor = {".pdf": parser}
documents = SimpleDirectoryReader(pdf_directory, file_extractor=file_extractor).load_data()

# bge-base embedding model
Settings.embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-base-en-v1.5")

# ollama
logging.basicConfig(level=logging.WARNING)
logging.getLogger('llama_index').setLevel(logging.ERROR)
logging.getLogger('httpx').setLevel(logging.ERROR)
logging.getLogger('httpcore').setLevel(logging.ERROR)

Settings.llm = Ollama(model="llama3", request_timeout=360.0)
index = VectorStoreIndex.from_documents(documents)

**Main Function to Handle Input and Query Processing**

In [ ]:
query_engine = index.as_query_engine()
question = ""    # your question here
print("\nQuestion: ", question)

try:
    response = query_engine.query(question)
    response_text = str(response)
    print("\nAnswer: ", response_text)

    # Write the output to a JSON file
    output_data = {"question": question, "answer": response_text}
    with open('output.json', 'w') as outfile:
        json.dump(output_data, outfile, indent=4)
except Exception as e:
    print(f"Error occurred: {e}")
    # Write the error to the output JSON file
    error_data = {"error": str(e)}
    with open('output.json', 'w') as outfile:
        json.dump(error_data, outfile, indent=4)